In [262]:
import cryptography
import hashlib
import os
import hmac
import base58
from bitstring import BitArray
from ecdsa import SigningKey, SECP256k1
from ecdsa.util import string_to_number, number_to_string

In [74]:
N_WORDS = 24
PASSPHRASE = ""
INITIAL_ENT = 256

with open("wordlist.txt", mode="r") as wordlist_file:
    wordlist = wordlist_file.read().split("\n")


In [79]:
def return_first_n_bits(n, bts):
    bits = BitArray(bts)
    return bits[:n].uint

def get_word_indices(wordlist_entropy):
    indices = []
    bits = BitArray(wordlist_entropy)
    i = 0
    while i < len(bits):
        indices.append(bits[i: i + 11].uint)
        i += 11
    return indices

# Bitcoin wallet generation demo

This Jupyter Notebook serves as a step-by-step demonstration of how a Bitcoin wallet is generated along with a bunch of public addresses.

__Disclaimer: You could, theoretically, use this as a way to generate your paper wallet, however this is not advised, since the code has not undergone any security review whatsover.__

## Generate Initial Entropy
Source: https://github.com/bitcoin/bips/blob/master/bip-0039.mediawiki

First we generate an initial entropy of `INITIAL_ENT / 8` bits and its checksum. By joining these we get an entropy that'll be used to generate the word list.

__ Notice, that we use `os.urandom` as our source of entropy. The [documentation states](https://docs.python.org/3/library/os.html#os.urandom) that this should be secure enough, but ultimately depends on your system configuration.__

In [80]:
initial_entropy = os.urandom(INITIAL_ENT // 8)
initial_digest = hashlib.sha256(initial_entropy).digest()
initial_checksum = return_first_n_bits(INITIAL_ENT // 32, initial_digest)
wordlist_entropy = initial_digest + bytes([initial_checksum])
print(f"Our wordlist entropy is {wordlist_entropy.hex()}")

Our wordlist entropy is 663dada0c37a7dca43108315129138483ee42293f6355a0e838880016011295666


In [81]:
word_indexes = get_word_indices(wordlist_entropy)
word_seed = [wordlist[i] for i in word_indexes]
print(f"Word seed is: {word_seed}")

Word seed is: ['great', 'uniform', 'habit', 'manage', 'pond', 'topple', 'arrange', 'away', 'bench', 'nest', 'evoke', 'motor', 'unusual', 'bacon', 'exist', 'shop', 'foam', 'inject', 'timber', 'abandon', 'bid', 'ancient', 'pipe', 'smoke']


We use the wordlist and an optional `PASSPHRASE` to generate the seed for the wallet key generation. The seed is generated using PBKDF2 key generation function.

In [82]:
seed = hashlib.pbkdf2_hmac("sha512", ("".join(word_seed) + PASSPHRASE).encode("utf-8"), ("mnemonic" + PASSPHRASE).encode("utf-8"), 2048)
print(f"Our seed is {seed.hex()}")

Our seed is bd76fbcf18d7fc1d095af458d6a94e6dba6fffa12bd3709145f3d66513251331584c55a137602c52e4fdcdd6a9d7888a8a009a6332fc16dfe24dea334f58007f


## Generating the wallet
Primary resource: https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki

Using the seed, we first generate the master key. We can subsequently use this key to generate child private and public keys for all our accounts.

In [425]:
# Definition of functions as presented in BIP-32. These include key derivation function as well as helper functions.

def serialise_bytes(key, key_type, chain_code, depth=0x00, fingerprint=bytes([0] * 4), child_no=bytes([0] * 4)):
    if key_type == "pub":
        version = 0x0488B21E.to_bytes(4, "big")
    elif key_type == "priv":
        version = 0x0488ADE4.to_bytes(4, "big")
        key = bytes([0]) + key.to_bytes(32, "big")
    return version + bytes([depth]) + fingerprint + child_no + chain_code + key

def key_to_b58(serialised_key):
    fingerprint = hashlib.sha256(hashlib.sha256(serialised_key).digest()).digest()[:4]
    return base58.b58encode(serialised_key + fingerprint)
    
def key_fingerprint(key):
    key = point_to_pk(key)
    ripemd = hashlib.new("ripemd160")
    ripemd.update(hashlib.sha256(key).digest())
    digest = ripemd.digest()
    return digest[:4]

def parse_256(p):
    return int.from_bytes(p, "big")

def point_to_pk(point):
    x = number_to_string(point.x(), SECP256k1.order)
    y = number_to_string(point.y(), SECP256k1.order)
    return bytes([((y[31] & 1) + 2)]) + x

def derive_key_and_chain(key, msg):
    digest = hmac.new(key=key, msg=msg, digestmod=hashlib.sha512).digest()
    return int.from_bytes(digest[:32], "big"), digest[32:]

def print_key_parts(key):
    print(key[:8])
    print(key[8:10])
    print(key[10:18])
    print(key[18:26])
    print(key[26:90])
    print(key[90:156])
    print(key[156:])
    
def point_mul(p):
    return SECP256k1.generator * p

def key_mod(key):
    return key % SECP256k1.order

# Derivation function definitions
def CKDpriv(key, chain, i):
    """Derives a child private key from a parent private key"""
    if i >= 2 ** 31:
        # we are generating a hardened key
        child_key, child_chain = derive_key_and_chain(chain, bytes([0]) + key.to_bytes(32, "big") + i.to_bytes(4, "big"))
    else:
        child_key, child_chain = derive_key_and_chain(chain, point_to_pk(point_mul(key)) + i.to_bytes(4, "big"))
    child_key = key_mod(child_key + key)
    return child_key, child_chain

def CKDpub(key, chain, i):
    if i >= 2 ** 31:
        raise ValueError("Derivation of pubkey from a parent pubkey is not defined")
    child_key, child_chain = derive_key_and_chain(chain, point_to_pk(key) + i.to_bytes(4, "big"))
    child_key = point_mul(int.from_bytes(child_key, "big")) + key
    return child_key, child_chain


In [426]:
seed = bytes.fromhex("000102030405060708090a0b0c0d0e0f") # this is a seed from the first test vector in BIP-32
master_key, master_chain = derive_key_and_chain(b"Bitcoin seed", seed)
print(f"Your master key is {master_key.to_bytes(32, 'big').hex()} and your master chain code is {master_chain.hex()}")

Your master key is e8f32e723decf4051aefac8e2c93c9c5b214313817cdb01a1494b917c8436b35 and your master chain code is 873dff81c02f525623fd1fe5167eac3a55a049de3d314bb42ee227ffed37d508


In [427]:
serialised_master = serialise_bytes(master_key, "priv", master_chain, fingerprint=bytes([0] * 4), child_no=bytes([0] * 4))
master_xpriv = key_to_b58(serialised_master)
print(f"Your master xpriv is {master_xpriv}")

Your master xpriv is xprv9s21ZrQH143K3QTDL4LXw2F7HEK3wJUD2nW2nRk4stbPy6cq3jPPqjiChkVvvNKmPGJxWUtg6LnF5kejMRNNU3TGtRBeJgk33yuGBxrMPHi


In [428]:
master_public_key = point_mul(master_key)
serialised_master_public = serialise_bytes(point_to_pk(master_public_key), "pub", master_chain, fingerprint=bytes([0] * 4), child_no=bytes([0] * 4))
print(f"Your master xpub is {key_to_b58(serialised_master_public)}")

Your master xpub is xpub661MyMwAqRbcFtXgS5sYJABqqG9YLmC4Q1Rdap9gSE8NqtwybGhePY2gZ29ESFjqJoCu1Rupje8YtGqsefD265TMg7usUDFdp6W1EGMcet8


This gives us the master public and private keys of our wallet. From these, we can subsequently generate our "sub-accounts". Examples bellow.

In [429]:
i = 2 ** 31
child_privk, child_chain = CKDpriv(master_key, master_chain, i)
serialised_child_priv = serialise_bytes(child_privk, "priv", child_chain, 1, key_fingerprint(master_public_key), i.to_bytes(4, "big"))
print(key_to_b58(serialised_child_priv))
key_to_b58(serialised_child_priv) == "xprv9uHRZZhk6KAJC1avXpDAp4MDc3sQKNxDiPvvkX8Br5ngLNv1TxvUxt4cV1rGL5hj6KCesnDYUhd7oWgT11eZG7XnxHrnYeSvkzY7d2bhkJ7"

xprv9uHRZZhk6KAJC1avXpDAp4MDc3sQKNxDiPvvkX8Br5ngLNv1TxvUxt4cV1rGL5hj6KCesnDYUhd7oWgT11eZG7XnxHrnYeSvkzY7d2bhkJ7


True

In [432]:
child_pub = point_mul(child_privk)
serialised_child_public = serialise_bytes(point_to_pk(child_pub), "pub", child_chain, 1, key_fingerprint(master_public_key), i.to_bytes(4, "big"))
print(key_to_b58(serialised_child_public))

xpub68Gmy5EdvgibQVfPdqkBBCHxA5htiqg55crXYuXoQRKfDBFA1WEjWgP6LHhwBZeNK1VTsfTFUHCdrfp1bgwQ9xv5ski8PX9rL2dZXvgGDnw


By applying CKDpriv and CKDpub recursively we generate the so-called _key tree_. The root of the tree is the master key, the leaf nodes are the actual wallet keys used for signing and verifying transactions and the inner nodes represent the intermediary parent keys. This structure allows for wallets that hold multiple accounts or even multiple coins (as long as they stick to the same key-derivation algorithms) without the need to generate multiple seeds.

A standard notation for this is of the following form:

```
m / i' / j / ... / k
```

where `m` is a placeholder for the root key and further components are the `i`th, `j`th (and so on) keys in the hierarchy. The apostrophe denotes a hardened key.

## Account hierarchy
Reference:
 * https://github.com/bitcoin/bips/blob/master/bip-0044.mediawiki
 * https://github.com/bitcoin/bips/blob/master/bip-0049.mediawiki

BIP-44 standarises the hierarchy in deterministic wallets.

```
m / purpose' / coin_type' / account' / change / address_index
```

The semantics behind most of the components are quite obvious from the name, maybe apart from `purpose`. `Purpose` determines the structure of the rest of the path. Usually it is set to `44` (as in BIP-44), however with the introduction of SegWit, constant `49` is used (as in BIP-49). Since we have to use hardened keys, the actual `int` representation will be `2^31 + 49`.

In [437]:
def to_hardened_int(i):
    return 2 ** 31 + i

PURPOSE = to_hardened_int(49)
COIN_TYPE = to_hardened_int(0)
ACCOUNT = to_hardened_int(0)